# `AzureContentSafetyTextTool`

>The `AzureContentSafetyTextTool` acts as a wrapper around the Azure AI Content Safety Service/API.
>The Tool will detect harmful content according to Azure's Content Safety Policy.

# Example

Get the required dependencies

In [1]:
import os

from langchain import hub

We will use a prompt to tell the model what to do. LangChain Prompts can be configured, however for sake of simplicity we will use a premade prompt from LangSmith. This requires an API key which can be setup [here](https://www.langchain.com/langsmith) after registration.

In [ ]:
LANGSMITH_KEY = os.environ["LANGSMITH_KEY"]
prompt = hub.pull("hwchase17/structured-chat-agent", api_key=LANGSMITH_KEY)

Now we can use the `AzureContentSafetyTextTool` combine with a model, using `create_structured_chat_agent`.

In [ ]:
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain_community.tools.azure_ai_services.content_safety import (
    AzureContentSafetyTextTool,
)
from langchain_openai import AzureChatOpenAI

In [ ]:
tools = [
    AzureContentSafetyTextTool(
        content_safety_key=os.environ["CONTENT_SAFETY_KEY"],
        content_safety_endpoint=os.environ["CONTENT_SAFETY_ENDPOINT"],
    )
]

model = AzureChatOpenAI(
    openai_api_version=os.environ["OPENAI_API_VERSION"],
    azure_deployment=os.environ["COMPLETIONS_MODEL"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
)

agent = create_structured_chat_agent(model, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

Then by using `.invoke`, the model can be told what to do and assess if using the tools it was given would assist in it's response.

In [ ]:
input = "I hate you"
agent_executor.invoke(
    {"input": f"Can you check the following text for harmful content : {input}"}
)